# Sklaidos matai statistikoje su Python: absoliutūs ir santykiniai

Šiame faile pateikiami dažniausi sklaidos matai, kurie padeda įvertinti, **kiek duomenys išsisklaido** aplink vidurkį ar medianą.
Naudojami paprasti verslo pavyzdžiai ir minimalūs duomenų rinkiniai, kad rezultatai būtų lengvai patikrinami.

## Sąvokos
- **Absoliutūs sklaidos matai** – matuojami tais pačiais vienetais kaip duomenys (pvz., Eur, vnt, min.).
- **Santykiniai sklaidos matai** – normalizuoti, dažniausiai išreiškiami procentais arba dalimi, todėl tinka palyginimams tarp skirtingų mastelių.

## Naudojamos bibliotekos
- `numpy` – pagrindiniai skaičiavimai
- `pandas` – duomenų lentelėms ir tvarkingam pateikimui
- `statistics` – kai kurie klasikiniai matai
- `scipy.stats` – IQR ir robustiški skaičiavimai (jei reikalinga)


In [ ]:
import numpy as np
import pandas as pd
import statistics as stats

from scipy import stats as scipy_stats

## Duomenys: paprastas verslo pavyzdys

Pavyzdinis rinkinys atspindi **dienos užsakymų sumas (Eur)** per dvi savaites.
Duomenyse įtrauktas vienas didesnis užsakymas, kad būtų matyti, kaip skirtingi matai reaguoja į išskirtis.


In [ ]:
orders_eur = pd.Series([
    120, 135, 128, 142, 150, 160, 155,
    130, 138, 145, 152, 149, 158, 420  # išskirtis (vienas labai didelis užsakymas)
], name="OrderValue_EUR")

orders_eur.describe()

## 1) Absoliutūs sklaidos matai (Absolute measures of dispersion)

### 1.1. Rėžis (Range)
Rėžis = max - min. Tai paprastas matas, bet labai jautrus išskirtims.


In [ ]:
data_min = orders_eur.min()
data_max = orders_eur.max()
data_range = data_max - data_min

data_min, data_max, data_range

### 1.2. Variacija (Variance) ir standartinis nuokrypis (Standard deviation)

- **Variacija** matuoja vidutinį kvadratinį nuokrypį nuo vidurkio.
- **Standartinis nuokrypis** yra variacijos kvadratinė šaknis ir turi tuos pačius vienetus kaip duomenys.

Svarbi detalė: dažniausiai analitikoje naudojamas **imties** (sample) standartinis nuokrypis su `ddof=1`.


In [5]:
mean_val = orders_eur.mean()

# Imties variacija ir standartinis nuokrypis (ddof=1)
var_sample = orders_eur.var(ddof=1)
std_sample = orders_eur.std(ddof=1)

# Populiacijos variacija ir standartinis nuokrypis (ddof=0)
var_pop = orders_eur.var(ddof=0)
std_pop = orders_eur.std(ddof=0)

mean_val, var_sample, std_sample, var_pop, std_pop

(163.0,
 5613.076923076923,
 74.92047065440075,
 5212.142857142857,
 72.19517197945342)

mean_val = 163.0  
Vidurkis. Vidutinė dienos užsakymo suma ≈ 163 Eur.

var_sample = 5613.08  
Imties variacija (ddof=1). Vidutinis kvadratinis nuokrypis nuo vidurkio. Vienetai – Eur².

std_sample = 74.92  
Imties standartinis nuokrypis (ddof=1). Tipinė dienos suma nuo vidurkio nukrypsta apie 75 Eur.

var_pop = 5212.14  
Populiacijos variacija (ddof=0). Mažesnė, nes laikoma, kad turimi visi duomenys.

std_pop = 72.20  
Populiacijos standartinis nuokrypis (ddof=0). Tipinis nuokrypis apie 72 Eur.

ddof paaiškinimas  
ddof (Delta Degrees of Freedom) nurodo, kiek laisvės laipsnių atimama skaičiuojant variaciją ar standartinį nuokrypį.  
ddof=1 naudojamas imčiai, kad būtų gautas nešališkas populiacijos sklaidos įvertinimas (Bessel korekcija).  
ddof=0 naudojamas populiacijai, kai laikoma, kad turimi visi galimi duomenys.


### 1.3. Vidutinis absoliutus nuokrypis (Mean Absolute Deviation, MAD nuo vidurkio)

Skaičiuojamas kaip vidutinis |x - vidurkis|. Šis matas yra mažiau „baudžiantis“ išskirtims nei variacija/standartinis nuokrypis.
Pandas nebeturi :( patogią funkciją `Series.mad()` (naudoja nuokrypį nuo vidurkio).


In [6]:
mad_mean = (orders_eur - orders_eur.mean()).abs().mean()
mad_mean

36.714285714285715

### 1.4. IQR – tarpkvartilinis rėžis (Interquartile Range)

IQR = Q3 - Q1. Tai robustiškas matas, mažiau jautrus išskirtims.


In [7]:
q1 = orders_eur.quantile(0.25)
q3 = orders_eur.quantile(0.75)
iqr = q3 - q1

q1, q3, iqr

(135.75, 154.25, 18.5)

## 2) Santykiniai sklaidos matai (Relative measures of dispersion)

Santykiniai matai leidžia palyginti sklaidą tarp skirtingų mastelių, pavyzdžiui:
- užsakymų sumos (Eur) ir
- užsakytų vienetų kiekis (vnt),
nes absoliutūs matai būtų skirtinguose vienetuose.

### 2.1. Variacijos koeficientas (Coefficient of Variation, CV)

CV = standartinis nuokrypis / vidurkis.
Dažnai pateikiamas procentais: CV% = CV * 100.

Pastaba: CV prasmingas, kai vidurkis yra teigiamas ir pakankamai toli nuo nulio.


In [8]:
cv = std_sample / mean_val
cv_pct = cv * 100

cv, cv_pct

(0.4596347892908021, 45.96347892908021)

### 2.2. Santykinis tarpkvartilinis rėžis (Relative IQR)

Vienas paprastas būdas normalizuoti IQR:
- Relative IQR = IQR / mediana

Tai ypač naudinga, kai lyginami skirstiniai su išskirtimis, nes IQR ir mediana yra robustiški.


In [9]:
median_val = orders_eur.median()
rel_iqr = iqr / median_val
rel_iqr_pct = rel_iqr * 100

median_val, rel_iqr, rel_iqr_pct

(147.0, 0.12585034013605442, 12.585034013605442)

### 2.3. Santykinis rėžis (Relative Range)

Relative Range = (max - min) / vidurkis.
Šis matas labai jautrus išskirtims, todėl praktikoje dažnai pakeičiamas robustiškesniais variantais (pvz., Relative IQR).


In [10]:
rel_range = data_range / mean_val
rel_range_pct = rel_range * 100

rel_range, rel_range_pct

(1.8404907975460123, 184.04907975460122)

## 3) Kodėl reikalingi ir absoliutūs, ir santykiniai matai

- Absoliutūs matai atsako į klausimą: **kiek vienetais** svyruoja reikšmės (pvz., Eur).
- Santykiniai matai atsako į klausimą: **kiek tai „didelė“ sklaida lyginant su lygiu** (pvz., vidurkiu ar mediana).

Toliau pateikiamas palyginimas su kitu rodikliu – **dienos užsakytų vienetų kiekiu**.


In [11]:
units = pd.Series([
    35, 40, 38, 42, 45, 44, 43,
    39, 41, 46, 47, 45, 44, 60  # kiek didesnė diena
], name="Units")

summary = pd.DataFrame({
    "Metric": ["OrderValue_EUR", "Units"],
    "Mean": [orders_eur.mean(), units.mean()],
    "Std (sample)": [orders_eur.std(ddof=1), units.std(ddof=1)],
    "CV%": [
        (orders_eur.std(ddof=1) / orders_eur.mean()) * 100,
        (units.std(ddof=1) / units.mean()) * 100
    ],
    "Median": [orders_eur.median(), units.median()],
    "IQR": [
        orders_eur.quantile(0.75) - orders_eur.quantile(0.25),
        units.quantile(0.75) - units.quantile(0.25)
    ],
    "Rel IQR% (IQR/Median)": [
        ((orders_eur.quantile(0.75) - orders_eur.quantile(0.25)) / orders_eur.median()) * 100,
        ((units.quantile(0.75) - units.quantile(0.25)) / units.median()) * 100
    ]
})

summary

,Metric,Mean,Std (sample),CV%,Median,IQR,Rel IQR% (IQR/Median)
0,OrderValue_EUR,163.0,74.920471,45.963479,147.0,18.50,12.585034
1,Units,43.5,5.814438,13.366525,43.5,4.75,10.919540


## 4) Išskirčių (outliers) įtaka ir robustiškumas

Išskirtys stipriai didina rėžį ir standartinį nuokrypį. IQR dažniausiai išlieka stabilesnis.

Toliau parodytas paprastas palyginimas: skaičiavimai su išskirtimi ir be jos.


In [12]:
orders_no_outlier = orders_eur[orders_eur < 300]  # paprastas filtras pagal verslo logiką

def dispersion_report(x: pd.Series) -> pd.Series:
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    mean = x.mean()
    std = x.std(ddof=1)
    return pd.Series({
        "n": x.size,
        "mean": mean,
        "std_sample": std,
        "range": x.max() - x.min(),
        "iqr": iqr,
        "cv_pct": (std / mean) * 100
    })

report = pd.DataFrame({
    "Su išskirtimi": dispersion_report(orders_eur),
    "Be išskirties": dispersion_report(orders_no_outlier)
})

report

,Su išskirtimi,Be išskirties
n,14.000000,13.000000
mean,163.000000,143.230769
std_sample,74.920471,12.383819
range,300.000000,40.000000
iqr,18.500000,17.000000
cv_pct,45.963479,8.646061


## 5) Dažnos klaidos ir gerosios praktikos

### Dažnos klaidos
- **Supainiojamas imties ir populiacijos standartinis nuokrypis** (`ddof=1` vs `ddof=0`).
- **CV taikomas, kai vidurkis yra 0 arba arti 0**, todėl gaunamas milžiniškas arba neapibrėžtas santykinis rodiklis.
- **Rėžis naudojamas kaip pagrindinis sklaidos matas**, ignoruojant išskirčių įtaką.
- **NaN reikšmės paliekamos skaičiavimuose**, todėl rezultatai gali tapti netikslūs arba klaidinantys.

### Gerosios praktikos
- Skaičiuojant standartinį nuokrypį imčiai naudoti `ddof=1` ir aiškiai tai nurodyti.
- Santykiniams palyginimams naudoti **CV** arba **Relative IQR**, priklausomai nuo išskirčių.
- Prieš skaičiavimus atlikti duomenų patikrą: trūkstamos reikšmės, nulinės reikšmės, keistos išskirtys.
- Validuoti svertinius ar sudėtinius skaičiavimus per paprastas formules (pvz., `sum / count`, `sum(value*weight)/sum(weight)`).


## 6) Greita atmintinė: kokį sklaidos matą rinktis

- **Range**: greita orientacija, bet labai jautrus išskirtims.
- **Std / Variance**: klasikiniai matai, naudingi modeliavimo kontekstuose, bet jautrūs išskirtims.
- **MAD (nuo vidurkio)**: paprastesnis ir mažiau jautrus nei std, bet vis tiek priklauso nuo vidurkio.
- **IQR**: robustiškas ir stabilus, dažnai naudingas verslo duomenyse su išskirtimis.
- **CV%**: leidžia lyginti sklaidą tarp skirtingų mastelių, bet netinka, kai vidurkis arti nulio.
- **Relative IQR%**: robustiškas santykinis matas, naudingas, kai yra išskirčių.


Laisvės laipsniai (degrees of freedom, df)

Laisvės laipsniai parodo, kiek nepriklausomų reikšmių lieka skaičiavimuose, kai dalis informacijos jau panaudota apribojimams, pvz., vidurkiui apskaičiuoti.

Jeigu apskaičiuojamas vidurkis, viena laisvė „dingsta“, nes paskutinė reikšmė nebegali būti parenkama laisvai.

Pavyzdys:
Jei yra n reikšmių ir skaičiuojamas vidurkis, lieka n − 1 laisvės laipsnių.

Ryšys su ddof:
ddof nurodo, kiek laisvės laipsnių atimama skaičiuojant variaciją ar standartinį nuokrypį.
ddof = 0 → df = n (populiacija)
ddof = 1 → df = n − 1 (imtis)

Laisvės laipsniai naudojami standartiniame nuokrypyje, t-teste, ANOVA, χ² teste ir pasikliautiniuose intervaluose.




Laisvės laipsniai parodo, kiek patikimi yra skaičiavimai, kai dirbama su ribotu duomenų kiekiu.

Jei skaičiuojamas vidurkis iš n reikšmių, viena reikšmė jau „panaudota“, todėl lieka n − 1 laisvės laipsnių.

Versle tai reiškia:
mažai duomenų → mažai laisvės laipsnių → didesnė paklaida ir rizika  
daug duomenų → daugiau laisvės laipsnių → stabilesni ir patikimesni rodikliai

Praktikoje:
ddof = 1 naudojamas, kai daromos išvados apie visą verslą iš dalinių duomenų (imtis).  
ddof = 0 naudojamas, kai skaičiuojami tik faktiniai duomenys be apibendrinimų (visa populiacija).
